<a href="https://colab.research.google.com/github/shihab1h/financial_timeseries_predictions/blob/main/langchain_extract_sentiment_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "XXXXX"

In [ ]:
!pip install langchain
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.7/993.7 kB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.4 MB/s eta 0:00:00


In [ ]:
from langchain.llms import OpenAI
import openai

# Run a simple prompt to ChatGPT

In [ ]:
# openai docs: https://github.com/openai/openai-python
models = openai.Model.list()
print([x.id for x in models['data']])

['whisper-1', 'babbage', 'davinci', 'text-davinci-edit-001', 'babbage-code-search-code', 'text-similarity-babbage-001', 'code-davinci-edit-001', 'text-davinci-001', 'ada', 'babbage-code-search-text', 'babbage-similarity', 'code-search-babbage-text-001', 'text-curie-001', 'code-search-babbage-code-001', 'text-ada-001', 'text-similarity-ada-001', 'curie-instruct-beta', 'ada-code-search-code', 'ada-similarity', 'text-embedding-ada-002', 'code-search-ada-text-001', 'text-search-ada-query-001', 'davinci-search-document', 'ada-code-search-text', 'text-search-ada-doc-001', 'davinci-instruct-beta', 'text-similarity-curie-001', 'code-search-ada-code-001', 'ada-search-query', 'text-search-davinci-query-001', 'curie-search-query', 'davinci-search-query', 'babbage-search-document', 'ada-search-document', 'text-search-curie-query-001', 'text-search-babbage-doc-001', 'gpt-3.5-turbo', 'text-davinci-003', 'curie-search-document', 'text-search-curie-doc-001', 'babbage-search-query', 'text-babbage-001',

In [ ]:
llm = OpenAI(temperature=0.9)

# Under the hood, large language models try to predict the next best word given a prompt. One word at a time. They assign a probability to each word in their vocabulary, and then picks a word among those.
# A temperature of 0 means roughly that the model will always select the highest probability word.
# A higher temperature means that the model might select a word with slightly lower probability, leading to more variation, randomness and creativity.
# A very high temperature therefore increases the risk of “hallucination”, meaning that the AI starts selecting words that will make no sense or be offtopic.


# Model list: https://platform.openai.com/docs/models/overview

In [ ]:
llm.model_name

'text-davinci-003'

In [ ]:
text = "What would be a good company name for a company that makes colorful socks?"
for i in range(10):
  print(llm(text))



Rainbow Socks Co.


Sox Splashes


PassionSox.


Rainbow Toe Tappers


HappySox.


Cheerful Socks Co.


SplashSox.


Happy Socks Co.


Rainbow Socks Company


ChromaCozy Socks.


# Prompt Templates: Manage prompts for LLMs
Calling an LLM is a great first step, but it’s just the beginning. Normally when you use an LLM in an application, you are not sending user input directly to the LLM. Instead, you are probably taking user input and constructing a prompt, and then sending that to the LLM.

For example, in the previous example, the text we passed in was hardcoded to ask for a name for a company that made colorful socks. In this imaginary service, what we would want to do is take only the user input describing what the company does, and then format the prompt with that information.

This is easy to do with LangChain!

First lets define the prompt template:

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

In [ ]:
print(prompt.format(product="adventurous pizzas"))

What is a good name for a company that makes adventurous pizzas?


In [ ]:
for i in range(10):
  print(llm(prompt.format(product="adventurous pizzas")))



Pizza Expedition.


Adventure Pizza Co.


Fire-Baked Adventures Pizzeria.


Adventure Pizza Co.


Pizza Trekkers.


PizzaQuest.


Pizza Expedition.


Pizza Odyssey.


Adventure Pizza Co.


Pizza Quest Adventures.


In [ ]:
import requests
from bs4 import BeautifulSoup

offset = 0
max_offset = 1000
offset_increment = 100

BASE_URL = 'http://www.forexrate.co.uk/'

In [ ]:
news_archive = []

for i in range(0,max_offset,offset_increment):
  url = f'http://www.forexrate.co.uk/newsarchive.php?start={i}'
  print(url)
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')
  tables = soup.findChildren('table')
  news_table = tables[1]
  rows = news_table.findChildren(['th', 'tr'])

  for idx,row in enumerate(rows):
    if idx == 0:
        continue
    cells = row.findChildren('td')
    for idx,cell in enumerate(cells):
      txt = cell.text
      href = cell.find('a')['href']
      href = BASE_URL + href.replace('./','')
      if "newsarchive.php?start=" in href:
        continue
      # let's get the date of the article
      date_page = requests.get(href)
      date_soup = BeautifulSoup(date_page.content, 'html.parser')
      date_div = date_soup.findChildren('div')[3]
      date_str = date_div.text
      news_archive.append({'txt':txt,'url':href,'date':date_str})
      print(len(news_archive), date_str, {'txt':txt,'url':href,'date':date_str})
      #print(value, href)
  #print(len(news_archive))

#print(news_archive)

http://www.forexrate.co.uk/newsarchive.php?start=0
1 06-06-2023 03:06 {'txt': '\nGold Price - Gold Price Attracting Longs Above $1,954, Focus on Key Events\n\t\t\tThe gold price turned to the upside after reaching yesterday’s low of $1,938. Now, the precious metal is trading at $1,962.\t\t', 'url': 'http://www.forexrate.co.uk/Gold-Price---Gold-Price-Attracting-Longs-Abov-news1686046914.html', 'date': '06-06-2023 03:06'}
2 06-06-2023 02:17 {'txt': '\nOil Price - FTSE 100 moves lower, fall in oil price weighs on BP and Shell\n\t\t\tFTSE 100 falls back, down 32 points Oil price falls, BP and Shell weaken Retail sales growth slows in May, BRC 10.17am: Half of multinationals plan to\t\t', 'url': 'http://www.forexrate.co.uk/Oil-Price---FTSE-100-moves-lower--fall-in-oil-news1686046916.html', 'date': '06-06-2023 02:17'}
3 05-06-2023 03:08 {'txt': "\nBrent Oil - Saudi Arabia's Solo Oil Production Cut Is a Risky Strategy\n\t\t\tPut together, both measures indicate that Moscow and Abu Dhabi are m

In [ ]:
import pandas as pd

df = pd.DataFrame(news_archive)

In [ ]:
df

,txt,url,date
0,\nGold Price - Gold Price Attracting Longs Abo...,http://www.forexrate.co.uk/Gold-Price---Gold-P...,06-06-2023 03:06
1,"\nOil Price - FTSE 100 moves lower, fall in oi...",http://www.forexrate.co.uk/Oil-Price---FTSE-10...,06-06-2023 02:17
2,\nBrent Oil - Saudi Arabia's Solo Oil Producti...,http://www.forexrate.co.uk/Brent-Oil---Saudi-A...,05-06-2023 03:08
3,\nEURUSD - FxWirePro- EURUSD Daily Outlook\n\t...,http://www.forexrate.co.uk/EURUSD---FxWirePro-...,05-06-2023 02:03
4,\nCommodities Prices - Commodity Price Bubble ...,http://www.forexrate.co.uk/Commodities-Prices-...,02-06-2023 02:45
...,...,...,...
115,\nBrent Oil - Oil rises after Opec+ agrees to ...,http://www.forexrate.co.uk/Brent-Oil---Oil-ris...,02-09-2021 02:57
116,\nGBPUSD - Forex technical analysis and foreca...,http://www.forexrate.co.uk/GBPUSD---Forex-tech...,02-09-2021 01:37
117,\nOil Price - Rice Bran Oil (CAS 68553-81-1) G...,http://www.forexrate.co.uk/Oil-Price---Rice-Br...,01-09-2021 03:15
118,\nCrude Oil - Commodity Wrap on September 1: N...,http://www.forexrate.co.uk/Crude-Oil---Commodi...,01-09-2021 02:50


In [ ]:
df["txt"][0]

'\nGold Price - Gold Price Attracting Longs Above $1,954, Focus on Key Events\n\t\t\tThe gold price turned to the upside after reaching yesterday’s low of $1,938. Now, the precious metal is trading at $1,962.\t\t'

In [ ]:
df["txt"].replace(to_replace= r'\n', value= '. ', regex=True, inplace=True)
df["txt"].replace(to_replace= r'\t', value= ' ', regex=True, inplace=True)

In [ ]:
df["txt"][0]

'. Gold Price - Gold Price Attracting Longs Above $1,954, Focus on Key Events.    The gold price turned to the upside after reaching yesterday’s low of $1,938. Now, the precious metal is trading at $1,962.  '

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df.to_csv('/content/drive/MyDrive/hist_fx.csv', index=False)

# Task

Evaluate sentiment in the news downloaded with your LLM, optimizing the prompt, and create a dataset that can be used with prophet as an extra regressor

In [ ]:
df

,txt,url,date
0,. Gold Price - Gold Price Attracting Longs Abo...,http://www.forexrate.co.uk/Gold-Price---Gold-P...,06-06-2023 03:06
1,". Oil Price - FTSE 100 moves lower, fall in oi...",http://www.forexrate.co.uk/Oil-Price---FTSE-10...,06-06-2023 02:17
2,. Brent Oil - Saudi Arabia's Solo Oil Producti...,http://www.forexrate.co.uk/Brent-Oil---Saudi-A...,05-06-2023 03:08
3,. EURUSD - FxWirePro- EURUSD Daily Outlook. ...,http://www.forexrate.co.uk/EURUSD---FxWirePro-...,05-06-2023 02:03
4,. Commodities Prices - Commodity Price Bubble ...,http://www.forexrate.co.uk/Commodities-Prices-...,02-06-2023 02:45
...,...,...,...
115,. Brent Oil - Oil rises after Opec+ agrees to ...,http://www.forexrate.co.uk/Brent-Oil---Oil-ris...,02-09-2021 02:57
116,. GBPUSD - Forex technical analysis and foreca...,http://www.forexrate.co.uk/GBPUSD---Forex-tech...,02-09-2021 01:37
117,. Oil Price - Rice Bran Oil (CAS 68553-81-1) G...,http://www.forexrate.co.uk/Oil-Price---Rice-Br...,01-09-2021 03:15
118,. Crude Oil - Commodity Wrap on September 1: N...,http://www.forexrate.co.uk/Crude-Oil---Commodi...,01-09-2021 02:50


In [ ]:
prompt_sentiment = PromptTemplate(
    input_variables=["txt"],
    template="Imagine you are an expert sentiment analyzer. Please classify the following news sentence as either positive, negative, or neutral: \"{txt}\"",
)

In [ ]:
def sentiment_classifier(txt):
  return llm(prompt_sentiment.format(txt=txt))

In [ ]:
df_sm = df.iloc[0:10]
df_sm

,txt,url,date,sentiment
0,. Gold Price - Gold Price Attracting Longs Abo...,http://www.forexrate.co.uk/Gold-Price---Gold-P...,06-06-2023 03:06,Unidentified
1,". Oil Price - FTSE 100 moves lower, fall in oi...",http://www.forexrate.co.uk/Oil-Price---FTSE-10...,06-06-2023 02:17,Unidentified
2,. Brent Oil - Saudi Arabia's Solo Oil Producti...,http://www.forexrate.co.uk/Brent-Oil---Saudi-A...,05-06-2023 03:08,Unidentified
3,. EURUSD - FxWirePro- EURUSD Daily Outlook. ...,http://www.forexrate.co.uk/EURUSD---FxWirePro-...,05-06-2023 02:03,Unidentified
4,. Commodities Prices - Commodity Price Bubble ...,http://www.forexrate.co.uk/Commodities-Prices-...,02-06-2023 02:45,Unidentified
5,. USDJPY - USD/JPY: Only a break below 138/137...,http://www.forexrate.co.uk/USDJPY---USD-JPY--O...,02-06-2023 01:42,Unidentified
6,. Crude Oil - Oil Prices Move Higher As U.S. H...,http://www.forexrate.co.uk/Crude-Oil---Oil-Pri...,01-06-2023 03:19,Unidentified
7,. Gold Price - Shining Surge: Gold prices soar...,http://www.forexrate.co.uk/Gold-Price---Shinin...,31-05-2023 03:21,Unidentified
8,. GBPUSD - GBP/USD Forecast: Sterling Looks To...,http://www.forexrate.co.uk/GBPUSD---GBP-USD-Fo...,31-05-2023 02:16,Unidentified
9,". Commodities Prices - Ukraine war, high commo...",http://www.forexrate.co.uk/Commodities-Prices-...,30-05-2023 02:40,Unidentified


In [ ]:
idf_sentiments = df_sm["txt"].apply(sentiment_classifier).replace(to_replace= r'\n\n', value= '', regex=True)

In [ ]:
df_sm["sentiment_llm"] = idf_sentiments
df_sm

<ipython-input-69-15cd482475d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sm["sentiment_llm"] = idf_sentiments


,txt,url,date,sentiment,sentiment_llm
0,. Gold Price - Gold Price Attracting Longs Abo...,http://www.forexrate.co.uk/Gold-Price---Gold-P...,06-06-2023 03:06,Unidentified,Neutral
1,". Oil Price - FTSE 100 moves lower, fall in oi...",http://www.forexrate.co.uk/Oil-Price---FTSE-10...,06-06-2023 02:17,Unidentified,Neutral
2,. Brent Oil - Saudi Arabia's Solo Oil Producti...,http://www.forexrate.co.uk/Brent-Oil---Saudi-A...,05-06-2023 03:08,Unidentified,Neutral
3,. EURUSD - FxWirePro- EURUSD Daily Outlook. ...,http://www.forexrate.co.uk/EURUSD---FxWirePro-...,05-06-2023 02:03,Unidentified,Neutral
4,. Commodities Prices - Commodity Price Bubble ...,http://www.forexrate.co.uk/Commodities-Prices-...,02-06-2023 02:45,Unidentified,Neutral
5,. USDJPY - USD/JPY: Only a break below 138/137...,http://www.forexrate.co.uk/USDJPY---USD-JPY--O...,02-06-2023 01:42,Unidentified,Neutral
6,. Crude Oil - Oil Prices Move Higher As U.S. H...,http://www.forexrate.co.uk/Crude-Oil---Oil-Pri...,01-06-2023 03:19,Unidentified,Neutral
7,. Gold Price - Shining Surge: Gold prices soar...,http://www.forexrate.co.uk/Gold-Price---Shinin...,31-05-2023 03:21,Unidentified,Positive
8,. GBPUSD - GBP/USD Forecast: Sterling Looks To...,http://www.forexrate.co.uk/GBPUSD---GBP-USD-Fo...,31-05-2023 02:16,Unidentified,Neutral
9,". Commodities Prices - Ukraine war, high commo...",http://www.forexrate.co.uk/Commodities-Prices-...,30-05-2023 02:40,Unidentified,Neutral
